In [6]:
import json
import sys
import os
import time
import logging
from collections import deque
from concurrent.futures import ThreadPoolExecutor
from typing import Dict, Optional

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as transforms
from sklearn.ensemble import RandomForestClassifier
import torch.optim.lr_scheduler as lr_scheduler

import tkinter as tk
from tkinter import ttk
import threading
import queue
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
import joblib
import copy
import random


In [7]:
# ==========================
# Configure Logging
# ==========================

logging.basicConfig(
    level=logging.INFO,
    format='[%(asctime)s] %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout),
        logging.FileHandler("training.log", mode='w')
    ]
)

In [8]:
# ==========================
# Model Definitions
# ==========================

class CNNMagician(nn.Module):
    def __init__(self, conv_layers=[(32, 5), (64, 5)], dropout_rate=0.5, use_batchnorm=True, num_classes=10):
        """
        Initialize CNNMagician model.

        Parameters:
        - conv_layers: List of tuples where each tuple defines (number_of_filters, kernel_size)
        - dropout_rate: Dropout probability to avoid overfitting
        - use_batchnorm: Boolean to decide whether to use batch normalization
        - num_classes: Number of output classes
        """
        super(CNNMagician, self).__init__()
        
        self.use_batchnorm = use_batchnorm
        self.conv_layers = nn.ModuleList()
        self.bns = nn.ModuleList()
        
        # Dynamically create convolutional layers
        in_channels = 1  # Input channels for grayscale images
        for out_channels, kernel_size in conv_layers:
            self.conv_layers.append(nn.Conv2d(in_channels, out_channels, kernel_size, padding=kernel_size // 2))
            if use_batchnorm:
                self.bns.append(nn.BatchNorm2d(out_channels))
            else:
                self.bns.append(nn.Identity())  # Skip batch normalization if not needed
            in_channels = out_channels

        self.pool = nn.MaxPool2d(2, 2)  # Pooling layer to reduce spatial dimensions
        self.fc1 = None  # Fully connected layer initialized dynamically
        self.fc2 = nn.Linear(128, num_classes)  # Final output layer
        self.dropout = nn.Dropout(dropout_rate)

        self.initialize_weights()  # Initialize weights for all layers

    def initialize_weights(self):
        """Apply Xavier initialization to convolutional and fully connected layers."""
        for module in self.modules():
            if isinstance(module, nn.Conv2d):
                nn.init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    nn.init.constant_(module.bias, 0)
            elif isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)
                nn.init.constant_(module.bias, 0)

    def preprocess_input(self, x):
        """
        Ensure the input tensor is correctly shaped and normalized.

        Parameters:
        - x: Input tensor
        """
        try:
            if isinstance(x, torch.Tensor) and len(x.shape) == 3:
                x = x.unsqueeze(1)  # Add a channel dimension for grayscale images
            elif isinstance(x, np.ndarray):
                x = torch.tensor(x, dtype=torch.float32).unsqueeze(1)  # Convert and add channel dimension
            x = x / 255.0  # Normalize input
            return x
        except Exception as e:
            logging.error(f"Error preprocessing input: {e}")
            raise

    def forward(self, x):
        """Forward pass through the CNNMagician."""
        try:
            x = self.preprocess_input(x)
            logging.info(f"Input shape after preprocessing: {x.shape}")
            
            # Apply convolutional layers with batch normalization and pooling
            for conv, bn in zip(self.conv_layers, self.bns):
                x = self.pool(F.relu(bn(conv(x))))
                logging.info(f"Shape after conv layer: {x.shape}")

            # Dynamically define fully connected layer based on the input size
            if self.fc1 is None:
                flattened_size = x.view(x.size(0), -1).size(1)
                self.fc1 = nn.Linear(flattened_size, 128).to(x.device)
                logging.info(f"Dynamically created fc1 with input size: {flattened_size}")

            # Flatten the tensor to match the input size of the fully connected layer
            x = x.view(x.size(0), -1)
            logging.info(f"Shape after flattening: {x.shape}")

            # Pass through the fully connected layers
            x = F.relu(self.fc1(x))
            logging.info(f"Shape after fc1: {x.shape}")
            x = self.dropout(x)  # Apply dropout
            x = self.fc2(x)  # Output layer
            return x

        except Exception as e:
            logging.error(f"Error in forward pass of CNNMagician: {e}")
            raise
        

class DNNMagician(nn.Module):
    def __init__(self, input_size=900, hidden_sizes=[512, 256], dropout_rate=0.5, num_classes=10, 
                 activation_fn=F.relu, use_layer_norm=False):
        """
        A flexible deep neural network with parameterized activation functions, normalization, 
        and dropout options.
        
        Parameters:
        - input_size (int): The size of the input feature vector.
        - hidden_sizes (list): List of sizes for hidden layers.
        - dropout_rate (float): Dropout rate applied after each hidden layer.
        - num_classes (int): The number of output classes.
        - activation_fn (function): Activation function to be used (default: ReLU).
        - use_layer_norm (bool): If True, use LayerNorm instead of BatchNorm.
        """
        super(DNNMagician, self).__init__()
        self.activation_fn = activation_fn
        self.use_layer_norm = use_layer_norm
        
        # Fully connected layers
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.fc3 = nn.Linear(hidden_sizes[1], num_classes)
        
        # Normalization layers
        if use_layer_norm:
            self.norm1 = nn.LayerNorm(hidden_sizes[0])
            self.norm2 = nn.LayerNorm(hidden_sizes[1])
        else:
            self.norm1 = nn.BatchNorm1d(hidden_sizes[0])
            self.norm2 = nn.BatchNorm1d(hidden_sizes[1])
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout_rate)

        # Initialize weights
        self._initialize_weights()

    def _initialize_weights(self):
        """Weight initialization for the fully connected layers."""
        nn.init.kaiming_normal_(self.fc1.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.fc2.weight, nonlinearity='relu')
        nn.init.xavier_uniform_(self.fc3.weight)

    def preprocess_input(self, x):
        """Preprocess input by ensuring it is flattened and normalized, with error handling for shape issues."""
        try:
            if isinstance(x, torch.Tensor):
                if len(x.shape) > 2:  # Flatten input if it's more than 2D
                    x = x.view(x.size(0), -1)
            else:
                raise ValueError("Input should be a torch.Tensor")
            x = x / 255.0  # Normalize input (assuming inputs are images in range [0, 255])
        except Exception as e:
            logging.error(f"Error in preprocessing input: {e}")
            raise
        return x

    def forward(self, x):
        # Preprocess input
        try:
            x = self.preprocess_input(x)
        except Exception as e:
            logging.error(f"Error during input preprocessing: {e}")
            return None

        # Layer 1 with normalization, activation, and dropout
        try:
            x = self.fc1(x)
            x = self.norm1(x)
            x = self.activation_fn(x)
            x = self.dropout(x)
        except Exception as e:
            logging.error(f"Error in forward pass at layer 1: {e}")
            return None
        
        # Layer 2 with normalization, activation, and dropout
        try:
            x = self.fc2(x)
            x = self.norm2(x)
            x = self.activation_fn(x)
            x = self.dropout(x)
        except Exception as e:
            logging.error(f"Error in forward pass at layer 2: {e}")
            return None
        
        # Final output layer (no activation)
        try:
            output = self.fc3(x)
        except Exception as e:
            logging.error(f"Error in forward pass at final layer: {e}")
            return None

        return output


class ResNetMagician(nn.Module):
    """
    Residual Neural Network architecture with input preprocessing and error handling for ARC dataset.
    """
    def __init__(self, num_classes=10, activation_fn=F.relu):
        """
        Parameters:
        - num_classes (int): The number of output classes.
        - activation_fn (function): Activation function to be used (default: ReLU).
        """
        super(ResNetMagician, self).__init__()
        self.activation_fn = activation_fn
        
        # Convolutional and batch normalization layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc = nn.Linear(64 * 7 * 7, num_classes)

        # Initialize weights
        self._initialize_weights()

    def _initialize_weights(self):
        """Initialize the weights of the layers."""
        nn.init.kaiming_normal_(self.conv1.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.conv2.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.conv3.weight, nonlinearity='relu')
        nn.init.xavier_uniform_(self.fc.weight)

    def preprocess_input(self, x):
        """Preprocess input by ensuring it has the correct shape and is normalized, with error handling."""
        try:
            if isinstance(x, torch.Tensor):
                if len(x.shape) == 3:
                    x = x.unsqueeze(1)  # Add channel dimension if missing (assuming grayscale input)
            elif isinstance(x, np.ndarray):
                x = torch.tensor(x, dtype=torch.float32).unsqueeze(1)  # Convert and add channel dimension
            else:
                raise ValueError("Input must be a torch.Tensor or a numpy array")
            x = x / 255.0  # Normalize input
        except Exception as e:
            logging.error(f"Error in input preprocessing: {e}")
            raise
        return x

    def forward(self, x):
        # Preprocess input
        try:
            x = self.preprocess_input(x)
        except Exception as e:
            logging.error(f"Error during input preprocessing: {e}")
            return None

        try:
            # First convolution + batch norm + activation
            residual = x  # Save input for residual connection
            out = self.activation_fn(self.bn1(self.conv1(x)))
        except Exception as e:
            logging.error(f"Error in forward pass at conv1: {e}")
            return None

        try:
            # Second convolution + batch norm + residual connection
            out = self.bn2(self.conv2(out))
            out += residual  # Add the residual connection
            out = self.activation_fn(out)
        except Exception as e:
            logging.error(f"Error in forward pass at conv2: {e}")
            return None
        
        try:
            # Third convolution + pooling
            out = self.pool(self.activation_fn(self.bn3(self.conv3(out))))
        except Exception as e:
            logging.error(f"Error in forward pass at conv3 and pooling: {e}")
            return None

        try:
            # Flatten and fully connected layer
            out = out.view(out.size(0), -1)
            out = self.fc(out)
        except Exception as e:
            logging.error(f"Error in forward pass at fully connected layer: {e}")
            return None

        return out

class VisionTransformerMagician(nn.Module):
    """
    Vision Transformer architecture with input preprocessing and error handling for ARC dataset.
    """
    def __init__(self, num_classes=10, patch_size=5, embed_dim=64, depth=6, num_heads=8, mlp_dim=128, dropout=0.1):
        super(VisionTransformerMagician, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer

        self.patch_size = patch_size
        self.embed_dim = embed_dim
        self.num_patches = (30 // patch_size) ** 2
        self.embedding = nn.Conv2d(1, embed_dim, kernel_size=patch_size, stride=patch_size)
        self.pos_embedding = nn.Parameter(torch.randn(1, self.num_patches, embed_dim))
        
        encoder_layers = TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dim_feedforward=mlp_dim, dropout=dropout)
        self.transformer = TransformerEncoder(encoder_layers, num_layers=depth)
        self.classifier = nn.Linear(embed_dim, num_classes)

        # Initialize weights
        self._initialize_weights()

    def _initialize_weights(self):
        """Initialize the weights of the model layers."""
        nn.init.kaiming_normal_(self.embedding.weight, nonlinearity='relu')
        nn.init.xavier_uniform_(self.classifier.weight)

    def preprocess_input(self, x):
        """Preprocess input to ensure it has the correct shape and normalization, with error handling."""
        try:
            if isinstance(x, np.ndarray):
                x = torch.tensor(x, dtype=torch.float32)
            if len(x.shape) == 3:  # If missing batch or channel dimension
                x = x.unsqueeze(1)  # Add channel dimension if missing (grayscale assumption)
            elif len(x.shape) == 4 and x.shape[1] != 1:  # Handle incorrect channel
                x = x[:, :1, :, :]  # Ensure single channel is processed
            x = x / 255.0  # Normalize input
        except Exception as e:
            logging.error(f"Error in input preprocessing: {e}")
            raise ValueError(f"Input preprocessing failed: {e}")
        return x

    def forward(self, x):
        # Preprocess input with error handling
        try:
            x = self.preprocess_input(x)
        except Exception as e:
            logging.error(f"Error during input preprocessing: {e}")
            return None
        
        try:
            # Apply patch embedding
            x = self.embedding(x)  # [batch_size, embed_dim, H/patch_size, W/patch_size]
        except Exception as e:
            logging.error(f"Error during embedding layer: {e}")
            return None
        
        try:
            x = x.flatten(2)  # Flatten height and width into a single patch dimension [batch_size, embed_dim, num_patches]
            x = x.transpose(1, 2)  # [batch_size, num_patches, embed_dim]
        except Exception as e:
            logging.error(f"Error during patch flattening or transpose: {e}")
            return None
        
        try:
            # Add positional embedding
            x = x + self.pos_embedding[:, :x.size(1), :]  # Match positional embedding size with x
        except Exception as e:
            logging.error(f"Error during positional embedding addition: {e}")
            return None
        
        try:
            # Pass through transformer encoder
            x = self.transformer(x)  # [batch_size, num_patches, embed_dim]
        except Exception as e:
            logging.error(f"Error during transformer encoder: {e}")
            return None
        
        try:
            # Global average pooling across patches
            x = x.mean(dim=1)  # [batch_size, embed_dim]
        except Exception as e:
            logging.error(f"Error during global average pooling: {e}")
            return None
        
        try:
            # Ensure correct shape for the classifier layer
            if x.size(1) != self.classifier.in_features:
                raise ValueError(f"Expected input size {self.classifier.in_features} but got {x.size(1)}")
        except Exception as e:
            logging.error(f"Error during classifier shape check: {e}")
            return None
        
        try:
            # Classification layer
            out = self.classifier(x)
        except Exception as e:
            logging.error(f"Error during final classifier layer: {e}")
            return None

        return out


In [9]:
# ==========================
# Data Loading and Preprocessing
# ==========================

def load_arc_data(file_paths: Optional[Dict[str, str]] = None, retries: int = 3, delay: float = 1.0) -> Dict[str, dict]:
    """
    Loads ARC dataset JSON files into a dictionary with enhanced error handling and retry logic.
    
    Parameters:
    - file_paths (dict): Optional. Dictionary of file names and paths. Defaults to ARC dataset.
    - retries (int): Number of times to retry loading a file if an error occurs.
    - delay (float): Delay in seconds between retries.

    Returns:
    - arc_data (dict): Dictionary containing the loaded ARC data.
    """
    default_file_paths = {
        "arc-agi_training-challenges": "arc-agi_training_challenges.json",
        "arc-agi_training-solutions": "arc-agi_training_solutions.json",
        "arc-agi_evaluation-challenges": "arc-agi_evaluation_challenges.json",
        "arc-agi_evaluation-solutions": "arc-agi_evaluation_solutions.json"
    }

    # Use custom file_paths if provided; otherwise, fall back to defaults
    file_paths = file_paths or default_file_paths
    arc_data = {}

    def load_single_file(key, path):
        """
        Loads a single file, with retry logic on failure.
        """
        if not os.path.exists(path):
            logging.error(f"File {path} does not exist. Please check the file path.")
            arc_data[key] = {}
            return

        for attempt in range(1, retries + 1):
            try:
                with open(path, 'r') as f:
                    arc_data[key] = json.load(f)
                    logging.info(f"Successfully loaded {key} from {path} on attempt {attempt}.")
                    return  # Exit on successful load
            except FileNotFoundError:
                logging.error(f"File {path} not found. Attempt {attempt} failed.")
            except json.JSONDecodeError:
                logging.error(f"File {path} contains invalid JSON. Attempt {attempt} failed.")
            except PermissionError:
                logging.error(f"Permission denied for file {path}.")
            except Exception as e:
                logging.error(f"An unexpected error occurred while loading {path}: {e}")

            # Retry logic if load fails
            if attempt < retries:
                logging.warning(f"Retrying {path} (attempt {attempt + 1}/{retries})...")
                time.sleep(delay)  # Delay between retries
            else:
                logging.error(f"Failed to load {path} after {retries} attempts.")
                arc_data[key] = {}  # Default empty value after max retries

    # Start loading process
    start_time = time.time()
    logging.info("Starting to load ARC dataset files...")

    # Use threading for concurrent file loading
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(load_single_file, key, path) for key, path in file_paths.items()]

        # Wait for all files to be loaded
        for future in futures:
            future.result()

    end_time = time.time()
    logging.info(f"Finished loading ARC dataset files. Total time: {end_time - start_time:.2f} seconds.")
    return arc_data

def pad_to_30x30(data):
    """
    Pads the input data to a 30x30 array.
    """
    if isinstance(data, list):
        data = np.array(data)  # Convert list to numpy array if necessary
    
    if not isinstance(data, (np.ndarray, torch.Tensor)):
        raise TypeError(f"Expected data to be of type np.ndarray or torch.Tensor, but got {type(data)}")
    
    padded_data = np.zeros((30, 30), dtype=data.dtype)
    padded_data[:data.shape[0], :data.shape[1]] = data
    return padded_data


def preprocess_arc_data(train_challenges, train_solutions):
    """
    Processes and pads input data to a consistent size (30x30) and prepares labels.
    """
    inputs = []
    labels = []
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomRotation(10),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))  # Example normalization
    ])
    
    for key, challenge in train_challenges.items():
        if 'train' in challenge:
            for item in challenge['train']:
                if 'input' in item and 'output' in item:
                    input_data = item['input']
                    label_data = item['output']
                    
                    # Ensure input_data and label_data are in expected types
                    if isinstance(input_data, (list, np.ndarray)) and isinstance(label_data, (list, np.ndarray)):
                        if input_data and label_data:
                            try:
                                padded_input = pad_to_30x30(input_data)
                                # Normalize to uint8
                                if padded_input.dtype != np.uint8:
                                    padded_input = (255 * (padded_input - padded_input.min()) / 
                                                    (padded_input.ptp() + 1e-8)).astype(np.uint8)

                                # Transform the input
                                padded_input = transform(padded_input)

                                # Append the numpy array of the padded input
                                inputs.append(padded_input.numpy())

                                # Simplify label: assuming one-hot encoding, take argmax
                                simplified_label = int(np.argmax(np.array(label_data).flatten()) % 10)  # Adjust as needed
                                labels.append(simplified_label)

                            except ValueError as value_error:
                                logging.warning(f"ValueError processing input data {input_data}: {value_error}")
                            except Exception as e:
                                logging.warning(f"Error processing item with input: {input_data} and label: {label_data}. Error: {e}")
                    else:
                        logging.warning(f"Invalid data type for input or output: input type {type(input_data)}, output type {type(label_data)}")

    # Validate that inputs and labels were populated
    if not inputs or not labels:
        raise ValueError("No valid data found during preprocessing.")

    inputs_array = np.array(inputs, dtype=np.float32).reshape(-1, 1, 30, 30)
    labels_array = np.array(labels, dtype=np.int64)
    logging.info(f"Preprocessed data: {inputs_array.shape[0]} samples.")
    
    return inputs_array, labels_array



def prepare_training_data(arc_data):
    """
    Converts processed data into PyTorch DataLoader for batching.
    
    Parameters:
    - arc_data: Dictionary containing processed ARC dataset.

    Returns:
    - DataLoader object for training data, or None if an error occurred.
    """
    try:
        train_challenges = arc_data['arc-agi_training-challenges']
        train_solutions = arc_data['arc-agi_training-solutions']
    except KeyError as e:
        logging.error(f"Missing key in arc_data: {e}")
        return None

    # Preprocess the data and handle potential errors
    try:
        input_data, labels = preprocess_arc_data(train_challenges, train_solutions)
    except ValueError as e:
        logging.error(f"Error during preprocessing: {e}")
        return None
    except Exception as e:
        logging.error(f"Unexpected error during preprocessing: {e}")
        return None

    # Check if input data and labels are not empty
    if input_data.size == 0 or labels.size == 0:
        logging.error("Input data or labels are empty after preprocessing.")
        return None

    # Create TensorDataset and DataLoader
    try:
        dataset = TensorDataset(torch.tensor(input_data, dtype=torch.float32), 
                                torch.tensor(labels, dtype=torch.int64))
        logging.info("Created TensorDataset for training.")
        
        data_loader = DataLoader(dataset, batch_size=32, shuffle=True)
        return data_loader
    except Exception as e:
        logging.error(f"Error creating DataLoader: {e}")
        return None
        
def prepare_evaluation_data(arc_data):
    """
    Prepares evaluation data similar to training data.

    Parameters:
    - arc_data: Dictionary containing processed ARC dataset.

    Returns:
    - DataLoader object for evaluation data, or None if an error occurred.
    """
    try:
        eval_challenges = arc_data['arc-agi_evaluation-challenges']
        eval_solutions = arc_data['arc-agi_evaluation-solutions']
    except KeyError as e:
        logging.error(f"Missing key in arc_data: {e}")
        return None

    # Preprocess the evaluation data and handle potential errors
    try:
        input_data, labels = preprocess_arc_data(eval_challenges, eval_solutions)
    except ValueError as e:
        logging.error(f"Error during preprocessing: {e}")
        return None
    except Exception as e:
        logging.error(f"Unexpected error during preprocessing: {e}")
        return None

    # Check if input data and labels are not empty
    if input_data.size == 0 or labels.size == 0:
        logging.error("Input data or labels are empty after preprocessing.")
        return None

    # Create TensorDataset and DataLoader
    try:
        dataset = TensorDataset(torch.tensor(input_data, dtype=torch.float32), 
                                torch.tensor(labels, dtype=torch.int64))
        logging.info("Created TensorDataset for evaluation.")
        
        data_loader = DataLoader(dataset, batch_size=32, shuffle=False)
        return data_loader
    except Exception as e:
        logging.error(f"Error creating DataLoader: {e}")
        return None

In [10]:
# ==========================
# Utility Functions
# ==========================

def generic_preprocessor(train_loader):
    """
    Generic preprocessor for models that don't require specific preprocessing steps.
    Applies normalization and reshaping if needed.

    Parameters:
    - train_loader: DataLoader object containing training data.

    Yields:
    - Normalized inputs and corresponding labels.
    """
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        try:
            # Ensure inputs are in float32 format and normalized to [0, 1]
            if not isinstance(inputs, torch.Tensor):
                raise TypeError(f"Expected inputs to be a torch.Tensor, got {type(inputs).__name__}")

            inputs = inputs.float() / 255.0
            
            # Check for required input shape
            if len(inputs.shape) < 2:
                raise ValueError(f"Input tensors must have at least 2 dimensions, but got {inputs.shape}.")

            yield inputs, labels

        except Exception as e:
            logging.error(f"Error in batch {batch_idx} during preprocessing: {e}")
            continue  # Proceed to the next batch even if one fails

def vision_transformer_preprocessor(train_loader, patch_size=5):
    """
    Preprocessor specifically for VisionTransformerMagician models.
    Ensures that the data is reshaped into patches.

    Parameters:
    - train_loader: DataLoader object containing training data.
    - patch_size: Size of the patches to be created from the images.

    Yields:
    - Reshaped inputs and corresponding labels.
    """
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        try:
            # Ensure inputs are in float32 format
            if not isinstance(inputs, torch.Tensor):
                raise TypeError(f"Expected inputs to be a torch.Tensor, got {type(inputs).__name__}")

            # Check for correct input dimensions
            if inputs.ndim != 4:
                raise ValueError(f"Expected input with 4 dimensions (batch_size, channels, height, width), but got {inputs.shape}.")

            # Reshape the input to match Vision Transformer input, assuming inputs are images
            inputs = inputs.unfold(2, patch_size, patch_size).unfold(3, patch_size, patch_size)
            inputs = inputs.contiguous().view(inputs.size(0), -1, patch_size * patch_size * inputs.size(1))  # Flatten patches
            
            # Normalize the inputs to [0, 1]
            inputs = inputs.float() / 255.0
            
            yield inputs, labels

        except Exception as e:
            logging.error(f"Error in batch {batch_idx} during Vision Transformer preprocessing: {e}")
            continue  # Proceed to the next batch even if one fails

def vision_transformer_preprocessor(train_loader, patch_size=5):
    """
    Preprocessor specifically for VisionTransformerMagician models.
    Ensures that the data is reshaped into patches.

    Parameters:
    - train_loader: DataLoader object containing training data.
    - patch_size: Size of the patches to be created from the images.

    Yields:
    - Reshaped inputs and corresponding labels.
    """
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        try:
            # Ensure inputs are a torch.Tensor
            if not isinstance(inputs, torch.Tensor):
                raise TypeError(f"Expected inputs to be a torch.Tensor, but got {type(inputs).__name__}.")

            # Check for correct input dimensions
            if inputs.ndim != 4:
                raise ValueError(f"Expected input with 4 dimensions (batch_size, channels, height, width), but got {inputs.shape}.")

            # Reshape the input to match Vision Transformer input, assuming inputs are images
            inputs = inputs.unfold(2, patch_size, patch_size).unfold(3, patch_size, patch_size)
            inputs = inputs.contiguous().view(inputs.size(0), -1, patch_size * patch_size * inputs.size(1))  # Flatten patches
            
            # Normalize the inputs to [0, 1]
            inputs = inputs.float() / 255.0
            
            yield inputs, labels

        except Exception as e:
            logging.error(f"Error in batch {batch_idx} during Vision Transformer preprocessing: {e}")
            continue  # Proceed to the next batch even if one fails


def cnn_preprocessor(train_loader):
    """
    Preprocessor specifically for CNNMagician models.
    Ensures that the data is in the correct format with the right dimensions.

    Parameters:
    - train_loader: DataLoader object containing training data.

    Yields:
    - Normalized inputs and corresponding labels.
    """
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        try:
            # Ensure inputs are a torch.Tensor
            if not isinstance(inputs, torch.Tensor):
                raise TypeError(f"Expected inputs to be a torch.Tensor, but got {type(inputs).__name__}.")

            # Check for correct input dimensions
            if inputs.ndim == 3:  # If the channel dimension is missing
                inputs = inputs.unsqueeze(1)  # Add channel dimension
            elif inputs.ndim != 4:
                raise ValueError(f"Expected input with 4 dimensions (batch_size, channels, height, width), but got {inputs.shape}.")

            # Normalize inputs to [0, 1]
            inputs = inputs.float() / 255.0
            
            yield inputs, labels

        except Exception as e:
            logging.error(f"Error in batch {batch_idx} during CNN preprocessing: {e}")
            continue  # Proceed to the next batch even if one fails

def dnn_preprocessor(train_loader):
    """
    Preprocessor specifically for DNNMagician models.
    Flattens the inputs for fully connected layers.

    Parameters:
    - train_loader: DataLoader object containing training data.

    Yields:
    - Normalized and flattened inputs and corresponding labels.
    """
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        try:
            # Ensure inputs are a torch.Tensor
            if not isinstance(inputs, torch.Tensor):
                raise TypeError(f"Expected inputs to be a torch.Tensor, but got {type(inputs).__name__}.")

            # Check for correct input dimensions
            if inputs.ndim < 2:  # If inputs are 1D or less
                raise ValueError(f"Expected input with at least 2 dimensions, but got {inputs.shape}.")
            
            # Flatten inputs for DNNs
            if inputs.ndim > 2:  # If inputs are not already flattened
                inputs = inputs.view(inputs.size(0), -1)

            # Normalize inputs to [0, 1]
            inputs = inputs.float() / 255.0

            yield inputs, labels

        except Exception as e:
            logging.error(f"Error in batch {batch_idx} during DNN preprocessing: {e}")
            continue  # Proceed to the next batch even if one fails
            
def resnet_preprocessor(train_loader):
    """
    Preprocessor specifically for ResNetMagician models.
    Ensures that the data is 4D (batch_size, channels, height, width).

    Parameters:
    - train_loader: DataLoader object containing training data.

    Yields:
    - Normalized inputs and corresponding labels.
    """
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        try:
            # Ensure inputs are a torch.Tensor
            if not isinstance(inputs, torch.Tensor):
                raise TypeError(f"Expected inputs to be a torch.Tensor, but got {type(inputs).__name__}.")

            # Check for correct input dimensions
            if inputs.ndim < 3:  # ResNet expects at least 3 dimensions (batch_size, height, width)
                raise ValueError(f"Expected input with at least 3 dimensions, but got {inputs.shape}.")
            
            # Ensure inputs are 4D: [batch_size, channels, height, width]
            if inputs.ndim == 3:  # If the channel dimension is missing, add it
                inputs = inputs.unsqueeze(1)  # Add the channel dimension
            
            # Normalize inputs to [0, 1]
            inputs = inputs.float() / 255.0

            yield inputs, labels

        except Exception as e:
            logging.error(f"Error in batch {batch_idx} during ResNet preprocessing: {e}")
            continue  # Proceed to the next batch even if one fails

def vision_transformer_postprocessor(model):
    """
    Postprocessor for VisionTransformerMagician models.
    Resets weights or parameters if necessary after training.

    Parameters:
    - model: The trained Vision Transformer model instance.
    """
    logging.info(f"Postprocessing Vision Transformer model {model.__class__.__name__}")

    try:
        # Example: Reset positional encodings if needed
        if hasattr(model, 'pos_embedding'):
            logging.info("Resetting positional embedding parameters.")
            model.pos_embedding = nn.Parameter(torch.randn_like(model.pos_embedding))

        # Optionally, reset other parameters or weights as needed
        for name, param in model.named_parameters():
            if 'weight' in name:
                logging.info(f"Resetting weights for {name}.")
                nn.init.kaiming_normal_(param)  # Reset weights using Kaiming normal initialization
            elif 'bias' in name:
                logging.info(f"Resetting biases for {name}.")
                nn.init.constant_(param, 0)  # Reset biases to zero

    except Exception as e:
        logging.error(f"Error during postprocessing of Vision Transformer model: {e}")

def cnn_postprocessor(model):
    """
    Postprocessor for CNNMagician models.
    Resets or reinitializes parts of the model if needed.

    Parameters:
    - model: The trained CNN model instance.
    """
    logging.info(f"Postprocessing CNN model {model.__class__.__name__}")

    try:
        # Example: Reset batch normalization layers if needed
        for layer in model.modules():
            if isinstance(layer, nn.BatchNorm2d):
                logging.info(f"Resetting running stats for {layer.__class__.__name__}.")
                layer.reset_running_stats()  # Reset running statistics

        # Optionally reinitialize weights of the convolutional layers
        for layer in model.modules():
            if isinstance(layer, nn.Conv2d):
                logging.info(f"Reinitializing weights for {layer.__class__.__name__}.")
                nn.init.kaiming_normal_(layer.weight, mode='fan_out', nonlinearity='relu')  # Kaiming normal initialization
                if layer.bias is not None:
                    nn.init.constant_(layer.bias, 0)  # Reset biases to zero

    except Exception as e:
        logging.error(f"Error during postprocessing of CNN model: {e}")

def dnn_postprocessor(model):
    """
    Postprocessor for DNNMagician models.
    Resets weights and biases after training if necessary.

    Parameters:
    - model: The trained DNN model instance.
    """
    logging.info(f"Postprocessing DNN model {model.__class__.__name__}")

    try:
        # Reset weights and biases in fully connected layers
        for layer in model.modules():
            if isinstance(layer, nn.Linear):
                logging.info(f"Resetting weights for layer {layer.__class__.__name__}.")
                nn.init.kaiming_normal_(layer.weight, mode='fan_in', nonlinearity='linear')  # Kaiming normal initialization
                if layer.bias is not None:
                    logging.info(f"Resetting biases for layer {layer.__class__.__name__}.")
                    nn.init.constant_(layer.bias, 0)  # Reset biases to zero

    except Exception as e:
        logging.error(f"Error during postprocessing of DNN model: {e}")

def resnet_postprocessor(model):
    """
    Postprocessor for ResNetMagician models.
    Resets layers and statistics after training if necessary.

    Parameters:
    - model: The trained ResNet model instance.
    """
    logging.info(f"Postprocessing ResNet model {model.__class__.__name__}")

    try:
        # Reset running stats for batch normalization layers
        for layer in model.modules():
            if isinstance(layer, nn.BatchNorm2d):
                logging.info(f"Resetting running stats for layer {layer.__class__.__name__}.")
                layer.reset_running_stats()

        # Example: Reinitialize the final fully connected layer
        if hasattr(model, 'fc'):
            logging.info(f"Reinitializing final layer {model.fc.__class__.__name__}.")
            nn.init.kaiming_normal_(model.fc.weight, mode='fan_out', nonlinearity='relu')  # Use appropriate initialization
            if model.fc.bias is not None:
                nn.init.constant_(model.fc.bias, 0)

    except Exception as e:
        logging.error(f"Error during postprocessing of ResNet model: {e}")


def ensure_correct_shape(x, expected_features):
    """
    Ensures that the input x has the correct shape to be passed into a layer that expects
    'expected_features' number of input features. Reshapes if necessary.

    Parameters:
    - x: The input tensor.
    - expected_features: The number of input features expected by the layer.

    Returns:
    - The reshaped tensor if necessary, or the original tensor if the shape is correct.
    """
    # Check if input is a tensor
    if not isinstance(x, torch.Tensor):
        logging.error("Input is not a PyTorch tensor.")
        raise ValueError("Input must be a PyTorch tensor.")

    # Log the current shape of the input
    logging.info(f"Current input shape: {x.shape}, expected shape features: {expected_features}")

    # Check if input shape matches expected features
    if x.size(1) != expected_features:
        # If not, print warning and reshape accordingly
        logging.warning(f"Input shape {x.size(1)} does not match expected {expected_features}. Attempting to reshape.")
        
        # Attempt reshaping the input
        try:
            # If the tensor has more than 2 dimensions, flatten the relevant dimensions
            if x.dim() > 2:
                x = x.view(x.size(0), -1)  # Flatten all but the batch dimension
                logging.info(f"Flattened input to shape: {x.shape}")
            else:
                x = x.view(x.size(0), expected_features)  # Adjust shape directly

        except Exception as e:
            logging.error(f"Error reshaping input tensor: {e}")
            raise RuntimeError(f"Could not reshape tensor from shape {x.shape} to expected shape with features {expected_features}.")

    return x

def ensure_correct_shape_for_model(inputs, model):
    """
    Adjusts the shape of the input tensor to fit the model's expected input.
    """
    if isinstance(model, CNNMagician) or isinstance(model, ResNetMagician):
        # CNN and ResNet expect 4D input: (batch_size, channels, height, width)
        if len(inputs.shape) == 2:
            # Reshape 2D tensor (batch_size, features) to 4D (batch_size, channels, height, width)
            # Assuming input image is square, we can infer height and width from sqrt of the feature size
            size = int(inputs.size(1) ** 0.5)  # Infer height/width
            inputs = inputs.view(inputs.size(0), 1, size, size)  # Reshape to (batch_size, 1, height, width)

    elif isinstance(model, DNNMagician):
        # DNN expects 2D input: (batch_size, features)
        if len(inputs.shape) == 4:
            # Flatten 4D tensor (batch_size, channels, height, width) to 2D (batch_size, features)
            inputs = inputs.view(inputs.size(0), -1)

    elif isinstance(model, VisionTransformerMagician):
        # Vision Transformer expects 4D input but with specific patches
        if len(inputs.shape) == 2:
            size = int(inputs.size(1) ** 0.5)
            inputs = inputs.view(inputs.size(0), 1, size, size)  # Reshape to (batch_size, 1, height, width)

    return inputs



def ensure_correct_shape_for_model(inputs, model):
    """
    Adjusts the shape of the input tensor to fit the model's expected input.

    Parameters:
    - inputs: The input tensor.
    - model: The model instance (e.g., CNNMagician, DNNMagician).

    Returns:
    - The reshaped input tensor.
    """
    # Log the current shape of the inputs
    logging.info(f"Current input shape: {inputs.shape} for model: {model.__class__.__name__}")

    try:
        if isinstance(model, (CNNMagician, ResNetMagician)):
            # CNN and ResNet expect 4D input: (batch_size, channels, height, width)
            if len(inputs.shape) == 2:
                # Reshape 2D tensor (batch_size, features) to 4D (batch_size, channels, height, width)
                size = int(inputs.size(1) ** 0.5)  # Infer height/width
                inputs = inputs.view(inputs.size(0), 1, size, size)  # Reshape to (batch_size, 1, height, width)
                logging.info(f"Reshaped inputs to 4D: {inputs.shape}")

        elif isinstance(model, DNNMagician):
            # DNN expects 2D input: (batch_size, features)
            if len(inputs.shape) == 4:
                # Flatten 4D tensor (batch_size, channels, height, width) to 2D (batch_size, features)
                inputs = inputs.view(inputs.size(0), -1)
                logging.info(f"Flattened inputs to 2D: {inputs.shape}")

        elif isinstance(model, VisionTransformerMagician):
            # Vision Transformer expects 4D input but with specific patches
            if len(inputs.shape) == 2:
                size = int(inputs.size(1) ** 0.5)
                inputs = inputs.view(inputs.size(0), 1, size, size)  # Reshape to (batch_size, 1, height, width)
                logging.info(f"Reshaped inputs for Vision Transformer to 4D: {inputs.shape}")

    except Exception as e:
        logging.error(f"Error while reshaping inputs for model {model.__class__.__name__}: {e}")
        raise RuntimeError("Input reshaping failed. Please check input dimensions and model expectations.")

    return inputs

def ensure_flattened(x, expected_size):
    """
    Ensures that the tensor is properly flattened to match the expected size.
    If not, it will attempt to flatten it until it matches the expected size.
    
    Args:
        x (torch.Tensor): The input tensor to flatten.
        expected_size (int): The expected size of the flattened tensor.
    
    Returns:
        torch.Tensor: The properly flattened tensor.

    Raises:
        ValueError: If the tensor cannot be flattened to the expected size.
    """
    # Log the initial shape of the input tensor
    logging.info(f"Initial shape of input tensor: {x.shape}")

    # Flatten the tensor
    flattened_size = x.view(x.size(0), -1).size(1)
    
    # Check if the current flattened size matches the expected size
    attempts = 0
    max_attempts = 10  # Safeguard to prevent infinite loops
    
    while flattened_size != expected_size and attempts < max_attempts:
        logging.warning(f"Flattened size {flattened_size} does not match expected size {expected_size}. Attempting to flatten again.")
        x = x.view(x.size(0), -1)  # Flatten the tensor
        flattened_size = x.size(1)  # Check the new flattened size
        attempts += 1
    
    if flattened_size != expected_size:
        logging.error(f"Unable to flatten tensor to expected size {expected_size} after {attempts} attempts.")
        raise ValueError(f"Tensor shape cannot be flattened to expected size {expected_size}. Current shape: {x.shape}")

    logging.info(f"Successfully flattened tensor to shape: {x.shape}")
    return x

def randomize_params():
    """
    Generates random hyperparameters for model training.
    
    Returns:
        dict: A dictionary containing the randomized hyperparameters.
    """
    try:
        num_models = random.randint(10, 20)
        num_epochs = random.randint(20, 200)
        initial_learning_rate = random.uniform(0.001, 0.01)  # Reasonable range for learning rates
        accuracy_threshold = random.uniform(0.75, 0.95)

        # Log the generated parameters
        logging.info(f"Randomized Parameters - Models: {num_models}, Epochs: {num_epochs}, "
                     f"LR: {initial_learning_rate:.4f}, Accuracy Threshold: {accuracy_threshold:.2f}")

        return {
            "num_models": num_models,
            "num_epochs": num_epochs,
            "initial_learning_rate": initial_learning_rate,
            "accuracy_threshold": accuracy_threshold
        }
    except Exception as e:
        logging.error(f"Error generating random parameters: {e}")
        raise
        
def get_random_model(model_type=None):
    """
    Instantiates a model based on the specified type.
    If no type is specified, randomly selects between available models.
    
    Args:
        model_type (str, optional): The type of model to instantiate. 
                                     If None, a model is chosen randomly.

    Returns:
        nn.Module: An instance of the specified or randomly chosen model.
    
    Raises:
        ValueError: If the model type is unknown.
    """
    available_models = ['CNN', 'DNN', 'ResNet', 'ViT']
    
    try:
        if model_type is None:
            model_type = random.choice(available_models)
        
        logging.info(f"Selected model type: {model_type}")

        if model_type == 'CNN':
            return CNNMagician()
        elif model_type == 'DNN':
            # Assuming input_size=900 for 30x30 images flattened
            return DNNMagician(input_size=900)
        elif model_type == 'ResNet':
            return ResNetMagician()
        elif model_type == 'ViT':
            return VisionTransformerMagician()
        else:
            raise ValueError(f"Unknown model type: {model_type}")

    except Exception as e:
        logging.error(f"Error instantiating model: {e}")
        raise

In [11]:
# ==========================
# Helper Reinforcement Model
# ==========================

class HyperparameterHelper:
    """
    A helper model to adjust hyperparameters based on training metrics.
    Uses a simple rule-based approach for demonstration, but can be expanded 
    with more sophisticated strategies if needed.
    """
    def __init__(self, initial_lr):
        self.lr = initial_lr
        self.lr_history = deque(maxlen=10)
        self.accuracy_history = deque(maxlen=10)
        self.loss_history = deque(maxlen=10)

    def update_metrics(self, loss, accuracy):
        """Update the recorded metrics for loss and accuracy."""
        self.loss_history.append(loss)
        self.accuracy_history.append(accuracy)

    def adjust_hyperparameters(self):
        """
        Adjust learning rate based on recent performance metrics.
        """
        if len(self.loss_history) < 10:
            return self.lr  # Not enough data to adjust

        avg_loss = np.mean(self.loss_history)
        avg_acc = np.mean(self.accuracy_history)

        # Simple rule-based adjustments
        try:
            if avg_loss < 0.5 and avg_acc > 0.9:
                self.lr *= 0.95  # Slightly decrease learning rate
                logging.info(f"Helper Model: Decreasing LR to {self.lr:.6f}")
            elif avg_loss > 1.0 and avg_acc < 0.5:
                self.lr *= 1.05  # Slightly increase learning rate
                logging.info(f"Helper Model: Increasing LR to {self.lr:.6f}")

            # Ensure learning rate remains within bounds
            self.lr = max(1e-6, min(self.lr, 1.0))

        except Exception as e:
            logging.error(f"Error adjusting hyperparameters: {e}")

        return self.lr

    def get_current_lr(self):
        """Return the current learning rate."""
        return self.lr

    def reset(self):
        """Reset the helper model metrics and learning rate."""
        self.lr_history.clear()
        self.accuracy_history.clear()
        self.loss_history.clear()
        self.lr = 0.001  # Reset to default or initial learning rate
        logging.info("Helper Model: Metrics reset and learning rate set to default.")

In [12]:
# ==========================
# GUI Class
# ==========================

class TrainingGUI:
    """
    A Tkinter-based GUI that displays real-time training progress, including current model number, epoch, loss, accuracy, and learning rate.
    """
    def __init__(self, root, total_models, total_epochs):
        self.root = root
        self.root.title("Model Training Progress Tracker")
        self.queue = queue.Queue()

        # Initialize GUI components
        self.model_label = tk.Label(root, text=f"Training Model: 0/{total_models}", font=("Helvetica", 14))
        self.model_label.pack(pady=5)

        self.epoch_label = tk.Label(root, text=f"Epoch: 0/{total_epochs}", font=("Helvetica", 14))
        self.epoch_label.pack(pady=5)

        self.loss_label = tk.Label(root, text="Loss: 0.0000", font=("Helvetica", 12))
        self.loss_label.pack(pady=2)

        self.accuracy_label = tk.Label(root, text="Accuracy: 0.0000", font=("Helvetica", 12))
        self.accuracy_label.pack(pady=2)

        self.val_loss_label = tk.Label(root, text="Validation Loss: 0.0000", font=("Helvetica", 12))
        self.val_loss_label.pack(pady=2)

        self.val_accuracy_label = tk.Label(root, text="Validation Accuracy: 0.0000", font=("Helvetica", 12))
        self.val_accuracy_label.pack(pady=2)

        self.lr_label = tk.Label(root, text="Learning Rate: 0.000000", font=("Helvetica", 12))
        self.lr_label.pack(pady=2)

        self.progress_bar = ttk.Progressbar(root, orient="horizontal", length=400, mode="determinate")
        self.progress_bar.pack(pady=10)

        # Real-time plots
        self.fig, self.ax = plt.subplots(figsize=(6, 4))
        self.line_loss, = self.ax.plot([], [], label='Training Loss', color='blue')
        self.line_val_loss, = self.ax.plot([], [], label='Validation Loss', color='orange')
        self.line_acc, = self.ax.plot([], [], label='Training Accuracy', color='green')
        self.line_val_acc, = self.ax.plot([], [], label='Validation Accuracy', color='red')
        self.ax.set_xlabel('Epochs')
        self.ax.set_ylabel('Metrics')
        self.ax.legend()
        self.ax.grid(True)
        self.canvas = FigureCanvasTkAgg(self.fig, master=root)
        self.canvas.draw()
        self.canvas.get_tk_widget().pack()

        self.loss_data = []
        self.val_loss_data = []
        self.acc_data = []
        self.val_acc_data = []

        # Ensemble Metrics
        self.ensemble_label = tk.Label(root, text="Ensemble Accuracy: N/A", font=("Helvetica", 14))
        self.ensemble_label.pack(pady=5)

        # Start processing the queue
        self.root.after(100, self.process_queue)

    def process_queue(self):
        """
        Process the queue for thread-safe GUI updates.
        """
        while not self.queue.empty():
            message = self.queue.get()
            if isinstance(message, dict):
                msg_type = message.get('type')
                if msg_type == 'epoch':
                    self.update_epoch(message)
                elif msg_type == 'mini_epoch':
                    self.update_mini_epoch(message)
                elif msg_type == 'ensemble_accuracy':
                    self.update_ensemble_accuracy(message.get('accuracy'))
                elif msg_type == 'training_completed':
                    self.model_label.config(text="Training Completed")
            else:
                logging.warning(f"Unexpected message type: {type(message)}")
        self.root.after(100, self.process_queue)

    def update_epoch(self, data):
        """
        Updates the GUI elements with new training epoch information.
        """
        try:
            self.model_label.config(text=f"Training Model: {data['model_num']}/{data['total_models']}")
            self.epoch_label.config(text=f"Epoch: {data['epoch']}/{data['total_epochs']}")
            self.loss_label.config(text=f"Loss: {data['loss']:.4f}")
            self.accuracy_label.config(text=f"Accuracy: {data['accuracy']:.4f}")
            self.val_loss_label.config(text=f"Validation Loss: {data['val_loss']:.4f}")
            self.val_accuracy_label.config(text=f"Validation Accuracy: {data['val_accuracy']:.4f}")
            self.lr_label.config(text=f"Learning Rate: {data['lr']:.6f}")

            # Update progress bar
            self.progress_bar["value"] = (data['epoch'] / data['total_epochs']) * 100
            self.root.update_idletasks()

            # Update plots
            self.loss_data.append(data['loss'])
            self.val_loss_data.append(data['val_loss'])
            self.acc_data.append(data['accuracy'])
            self.val_acc_data.append(data['val_accuracy'])

            self.line_loss.set_data(range(1, len(self.loss_data) + 1), self.loss_data)
            self.line_val_loss.set_data(range(1, len(self.val_loss_data) + 1), self.val_loss_data)
            self.line_acc.set_data(range(1, len(self.acc_data) + 1), self.acc_data)
            self.line_val_acc.set_data(range(1, len(self.val_acc_data) + 1), self.val_acc_data)

            self.ax.relim()
            self.ax.autoscale_view()
            self.canvas.draw()
        except Exception as e:
            logging.error(f"Error updating epoch data: {e}")

    def update_mini_epoch(self, data):
        """
        Updates the GUI elements with new injected mini-epoch information.
        """
        try:
            self.model_label.config(text=f"Training Model: {data['model_num']}/{data['total_models']}")
            self.epoch_label.config(text=f"Injected Mini-Epoch: {data['current_mini_epoch']}/{data['max_mini_epochs']}")
            self.loss_label.config(text=f"Loss: {data['loss']:.4f}")
            self.accuracy_label.config(text=f"Accuracy: {data['accuracy']:.4f}")
            self.val_loss_label.config(text=f"Validation Loss: {data['val_loss']:.4f}")
            self.val_accuracy_label.config(text=f"Validation Accuracy: {data['val_accuracy']:.4f}")
            self.lr_label.config(text=f"Learning Rate: {data['lr']:.6f}")

            # Update plots
            self.loss_data.append(data['loss'])
            self.val_loss_data.append(data['val_loss'])
            self.acc_data.append(data['accuracy'])
            self.val_acc_data.append(data['val_accuracy'])

            self.line_loss.set_data(range(1, len(self.loss_data) + 1), self.loss_data)
            self.line_val_loss.set_data(range(1, len(self.val_loss_data) + 1), self.val_loss_data)
            self.line_acc.set_data(range(1, len(self.acc_data) + 1), self.acc_data)
            self.line_val_acc.set_data(range(1, len(self.val_acc_data) + 1), self.val_acc_data)

            self.ax.relim()
            self.ax.autoscale_view()
            self.canvas.draw()
        except Exception as e:
            logging.error(f"Error updating mini epoch data: {e}")

    def update_ensemble_accuracy(self, accuracy):
        """
        Updates the ensemble accuracy label.
        """
        try:
            self.ensemble_label.config(text=f"Ensemble Accuracy: {accuracy:.4f}")
        except Exception as e:
            logging.error(f"Error updating ensemble accuracy: {e}")


In [13]:
# ==========================
# Training Functions
# ==========================

def ensure_correct_shape(x, expected_shape):
    """
    Ensure the input tensor matches the expected shape by flattening or reshaping if necessary.

    Args:
        x (torch.Tensor): The input tensor to reshape.
        expected_shape (int): The expected number of input features.

    Returns:
        torch.Tensor: The reshaped tensor.
    """
    # Check if the input is a tensor
    if not isinstance(x, torch.Tensor):
        raise TypeError("Input must be a PyTorch tensor.")

    original_shape = x.shape  # Store the original shape for logging
    actual_shape = x.size(1)  # Input feature size (excluding batch dimension)

    if actual_shape != expected_shape:
        logging.warning(f"Shape mismatch: Expected {expected_shape}, got {actual_shape}. Adjusting...")
        
        # If the tensor has more than 2 dimensions, we need to flatten it
        if len(original_shape) > 2:  
            x = x.view(x.size(0), -1)  # Flatten while preserving batch size
        
        # Check again if the flattened shape matches the expected shape
        actual_shape = x.size(1)
        if actual_shape != expected_shape:
            logging.error(f"Failed to reshape tensor. Original shape: {original_shape}. Current shape: {x.shape}.")
            raise RuntimeError(f"Unable to reshape tensor to match the expected shape {expected_shape}. Got {actual_shape} instead.")

    logging.info(f"Input tensor reshaped from {original_shape} to {x.shape}.")
    return x



def evaluate_model(model, eval_loader, criterion, device):
    """
    Evaluates the model on the evaluation data, ensuring inputs are preprocessed.
    
    Args:
        model (nn.Module): The model to evaluate.
        eval_loader (DataLoader): The DataLoader containing evaluation data.
        criterion (nn.Module): The loss function to use for evaluation.
        device (torch.device): The device to run the evaluation on.

    Returns:
        tuple: Average loss and accuracy of the model on the evaluation data.
    """
    model.eval()
    total_loss = 0
    correct_predictions = 0
    total_samples = 0

    if len(eval_loader) == 0:
        logging.warning("Evaluation loader is empty. No evaluation performed.")
        return float('inf'), 0.0  # Return high loss and zero accuracy

    with torch.no_grad():
        for inputs, labels in eval_loader:
            try:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # Ensure the model preprocesses inputs
                outputs = model(inputs)
                
                loss = criterion(outputs, labels)
                total_loss += loss.item()
                
                _, predicted = torch.max(outputs.data, 1)
                correct_predictions += (predicted == labels).sum().item()
                total_samples += labels.size(0)

            except Exception as e:
                logging.error(f"Error during evaluation: {e}")
                continue  # Log the error and skip this batch

    avg_loss = total_loss / len(eval_loader) if total_samples > 0 else float('inf')
    accuracy = correct_predictions / total_samples if total_samples > 0 else 0.0

    logging.info(f"Evaluation complete. Average Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")
    return avg_loss, accuracy

def train_regular_model(model, train_loader, eval_loader, num_epochs, initial_learning_rate, gui, model_num, total_models, helper, device):
    """
    Trains a single model (CNN, DNN, ResNet, ViT, etc.) with adaptive learning rate and early stopping.
    """
    logging.debug(f"Starting training for Model {model_num} with architecture: {model.__class__.__name__}")

    model.to(device)  # Move model to the appropriate device
    optimizer = Adam(model.parameters(), lr=initial_learning_rate)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(1, num_epochs + 1):
        logging.debug(f"Epoch {epoch}/{num_epochs} for Model {model_num}")
        
        # Training loop
        for inputs, labels in train_loader:
            # Ensure inputs and labels are moved to the device
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update weights

            logging.debug(f"Model {model_num}, Epoch {epoch}: Loss = {loss.item():.4f}")

    logging.info(f"Completed training for Model {model_num}.")




def inject_mini_epochs(model, train_loader, eval_loader, optimizer, scheduler, criterion,
                      gui, model_num, total_models, helper, best_loss, patience=10, 
                      max_injections=7, epochs_per_injection=3, current_layer=1, max_layers=7):
    """
    Injects additional mini-epochs with different model architectures to overcome stagnation until improvement is found.
    Dynamically reshapes data and adapts architecture to ensure compatibility without breaking or skipping any steps.
    """
    def create_mini_model(layer):
        """Instantiate a mini model based on the current layer."""
        if layer == 1:
            return DNNMagician(input_size=900)
        elif layer == 2:
            return ResNetMagician()
        elif layer == 3:
            return VisionTransformerMagician()
        elif layer == 4:
            return CNNMagician()
        elif layer in [5, 6, 7]:
            return DNNMagician(input_size=900) if layer % 2 == 0 else ResNetMagician()
        return CNNMagician()  # Default fallback

    def train_mini_model(mini_model, train_loader):
        """Train the mini model for a specified number of epochs."""
        mini_model.train()
        total_loss, correct_predictions, total_samples = 0, 0, 0

        for inputs, labels in train_loader:
            try:
                inputs, labels = inputs.to(device), labels.to(device)
                inputs = try_dynamic_reshape(inputs, mini_model)
                
                # Forward pass
                outputs = mini_model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                mini_optimizer.step()

                total_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                correct_predictions += (predicted == labels).sum().item()
                total_samples += labels.size(0)

            except (RuntimeError, Exception) as error:
                logging.error(f"Error during training mini model: {error}")
                continue  # Log and proceed to the next batch

        avg_loss = total_loss / len(train_loader) if total_samples > 0 else float('inf')
        accuracy = correct_predictions / total_samples if total_samples > 0 else 0.0
        return avg_loss, accuracy

    injected_epochs = 0
    improvement_found = False

    while injected_epochs < max_injections and not improvement_found:
        injected_epochs += 1
        logging.info(f"Model {model_num}/{total_models}, Injected Mini-Epoch [{injected_epochs}/{max_injections}] at Layer {current_layer}/{max_layers}")

        mini_model = create_mini_model(current_layer)
        mini_model.to(device)
        mini_optimizer = Adam(mini_model.parameters(), lr=optimizer.param_groups[0]['lr'])
        mini_scheduler = lr_scheduler.ReduceLROnPlateau(mini_optimizer, mode='min', patience=5, factor=0.5)
        mini_criterion = nn.CrossEntropyLoss()
        mini_helper = copy.deepcopy(helper)  # Clone the helper to maintain separate histories

        for mini_epoch in range(1, epochs_per_injection + 1):
            avg_loss, accuracy = train_mini_model(mini_model, train_loader)

            # Validation Phase
            try:
                injected_val_loss, injected_val_accuracy = evaluate_model(mini_model, eval_loader, mini_criterion, device)
                mini_scheduler.step(injected_val_loss)
            except Exception as eval_error:
                logging.error(f"Error during validation in mini model: {eval_error}")
                injected_val_loss, injected_val_accuracy = float('inf'), 0.0

            logging.info(f"Model {model_num}/{total_models}, Injected Mini-Epoch [{injected_epochs}/{max_injections}], "
                         f"Layer: {current_layer}, Epoch [{mini_epoch}/{epochs_per_injection}], "
                         f"Model Type: {mini_model.__class__.__name__}, "
                         f"Training Loss: {avg_loss:.4f}, Training Accuracy: {accuracy:.4f}, "
                         f"Validation Loss: {injected_val_loss:.4f}, Validation Accuracy: {injected_val_accuracy:.4f}")

            # Update GUI
            try:
                gui.queue.put({
                    'type': 'mini_epoch',
                    'model_num': model_num,
                    'total_models': total_models,
                    'current_mini_epoch': injected_epochs,
                    'max_mini_epochs': max_injections,
                    'loss': avg_loss,
                    'accuracy': accuracy,
                    'val_loss': injected_val_loss,
                    'val_accuracy': injected_val_accuracy,
                    'lr': mini_optimizer.param_groups[0]['lr']
                })
            except Exception as gui_error:
                logging.error(f"Error updating GUI with mini-epoch info: {gui_error}")

            # Update helper model and adjust learning rate
            try:
                mini_helper.update_metrics(injected_val_loss, injected_val_accuracy)
                adjusted_mini_lr = mini_helper.adjust_hyperparameters()
                for param_group in mini_optimizer.param_groups:
                    param_group['lr'] = adjusted_mini_lr
            except Exception as helper_error:
                logging.error(f"Error adjusting hyperparameters or updating helper model: {helper_error}")

            # Check for improvement
            if injected_val_loss < best_loss:
                best_loss = injected_val_loss
                improvement_found = True
                patience = 10  # Reset patience after improvement
                logging.info(f"Improvement found during injected mini-epochs for Model {model_num}.")
                try:
                    torch.save(mini_model.state_dict(), f'best_model_{model_num}_mini_epoch_layer{current_layer}.pth')
                except Exception as save_error:
                    logging.error(f"Error saving mini-epoch model at Layer {current_layer}: {save_error}")

        # Proceed to next layer if no improvement found
        if not improvement_found and current_layer < max_layers:
            current_layer += 1  # Move to the next layer

def try_dynamic_reshape(inputs: torch.Tensor, model: nn.Module) -> torch.Tensor:
    """
    Dynamically reshapes inputs to match the expected shape of the model.
    Tries to flatten, reduce, or modify the input data to ensure it can be processed
    by the model without shape errors. Includes error handling and corrective steps.

    Args:
        inputs (torch.Tensor): The input tensor to reshape.
        model (nn.Module): The model to which the inputs will be passed.

    Returns:
        torch.Tensor: The reshaped inputs compatible with the model.
    """
    def reshape_for_dnn(inputs: torch.Tensor, model: nn.Module) -> torch.Tensor:
        """Reshape inputs for fully connected models."""
        inputs = inputs.view(inputs.size(0), -1)  # Flatten inputs
        input_size = model.fc1.in_features
        if inputs.size(1) != input_size:
            logging.warning(f"Input size mismatch for DNN: {inputs.size(1)} vs. {input_size}. Adjusting size.")
            if inputs.size(1) > input_size:
                inputs = inputs[:, :input_size]  # Trim the input
            else:
                # Pad the input if it's smaller than expected
                padding = torch.zeros(inputs.size(0), input_size - inputs.size(1)).to(inputs.device)
                inputs = torch.cat((inputs, padding), dim=1)
        return inputs

    def reshape_for_cnn(inputs: torch.Tensor, model: nn.Module) -> torch.Tensor:
        """Reshape inputs for CNN and ResNet models."""
        expected_channels = model.conv1.in_channels
        if inputs.dim() == 2:
            height = int(inputs.size(1) ** 0.5)
            inputs = inputs.view(inputs.size(0), 1, height, height)  # Assuming square input images
        if inputs.size(1) != expected_channels:
            logging.warning(f"Adjusting input channels from {inputs.size(1)} to {expected_channels}.")
            inputs = inputs.unsqueeze(1)  # Add channel dimension if necessary
        return inputs

    def reshape_for_transformer(inputs: torch.Tensor, model: nn.Module) -> torch.Tensor:
        """Reshape inputs for Transformer models."""
        patch_size = model.patch_size
        inputs = inputs.unfold(2, patch_size, patch_size).unfold(3, patch_size, patch_size)
        inputs = inputs.flatten(2).transpose(1, 2)  # Transform input into patches
        return inputs

    try:
        if hasattr(model, 'fc1'):  # DNNMagician
            inputs = reshape_for_dnn(inputs, model)
        elif hasattr(model, 'conv1'):  # CNNMagician or ResNetMagician
            inputs = reshape_for_cnn(inputs, model)
        elif hasattr(model, 'embedding'):  # VisionTransformerMagician
            inputs = reshape_for_transformer(inputs, model)
        else:
            logging.error(f"Model type not recognized for reshaping: {model.__class__.__name__}")

    except RuntimeError as reshape_error:
        logging.error(f"Error reshaping inputs: {reshape_error}")
        try:
            inputs = inputs.view(inputs.size(0), -1)  # Last resort: fully flatten inputs
            logging.info(f"Falling back to fully flattened input: {inputs.shape}")
        except Exception as fallback_error:
            logging.error(f"Failed to reshape even after fallback: {fallback_error}")
            raise fallback_error

    return inputs

def combine_models_random_forest(train_loader, regular_models, mini_epoch_models=None, n_estimators=100, random_state=42):
    """
    Combines outputs from multiple models (regular and mini-epoch) using a Random Forest classifier.

    Args:
        train_loader: DataLoader providing training data.
        regular_models: List of regular trained models.
        mini_epoch_models: Optional list of mini-epoch models.
        n_estimators: Number of trees in the Random Forest (default: 100).
        random_state: Random state for reproducibility (default: 42).

    Returns:
        RandomForestClassifier: The trained Random Forest classifier.
    """
    if mini_epoch_models is None:
        mini_epoch_models = []
        
    all_train_features = []
    all_train_labels = []

    for batch_idx, (inputs, labels) in enumerate(train_loader):
        model_outputs = []
        for model in regular_models + mini_epoch_models:
            try:
                with torch.no_grad():
                    outputs = model(inputs)
                    model_outputs.append(outputs.detach().cpu().numpy())
            except Exception as e:
                logging.error(f"Error during model prediction in batch {batch_idx}: {e}")
                continue  # Skip this model if there's an error

        if not model_outputs:
            logging.warning(f"No outputs collected for batch {batch_idx}. Skipping...")
            continue

        features = np.concatenate(model_outputs, axis=1)  # Concatenate model outputs along the feature axis
        all_train_features.append(features)
        all_train_labels.append(labels.cpu().numpy())

    if not all_train_features or not all_train_labels:
        raise ValueError("No features or labels collected from models. Cannot proceed with training.")

    try:
        rf = RandomForestClassifier(n_estimators=n_estimators, random_state=random_state)
        rf.fit(np.vstack(all_train_features), np.hstack(all_train_labels))
        logging.info("Combined models using Random Forest.")
    except Exception as e:
        logging.error(f"Error fitting Random Forest model: {e}")
        raise

    return rf

def evaluate_ensemble_model(ensemble_model, eval_loader, regular_models, mini_epoch_models=None):
    """
    Evaluates the ensemble model's performance on evaluation data.

    Args:
        ensemble_model: The trained ensemble model (e.g., Random Forest).
        eval_loader: DataLoader providing evaluation data.
        regular_models: List of regular trained models.
        mini_epoch_models: Optional list of mini-epoch models.

    Returns:
        predictions: The predicted labels for the evaluation dataset.
        all_eval_labels: The true labels for the evaluation dataset.
        accuracy: The accuracy of the ensemble model.
    """
    if mini_epoch_models is None:
        mini_epoch_models = []
        
    all_eval_features, all_eval_labels = [], []

    for batch_idx, (inputs, labels) in enumerate(eval_loader):
        model_outputs = []
        for model in regular_models + mini_epoch_models:
            try:
                with torch.no_grad():
                    outputs = model(inputs)
                    model_outputs.append(outputs.detach().cpu().numpy())
            except Exception as e:
                logging.error(f"Error during model prediction in batch {batch_idx}: {e}")
                continue  # Skip this model if there's an error

        if not model_outputs:
            logging.warning(f"No outputs collected for batch {batch_idx}. Skipping...")
            continue

        # Concatenate model outputs along the feature axis
        features = np.concatenate(model_outputs, axis=1)
        all_eval_features.append(features)
        all_eval_labels.append(labels.cpu().numpy())

    if not all_eval_features or not all_eval_labels:
        raise ValueError("No features or labels collected from models. Cannot proceed with evaluation.")

    try:
        predictions = ensemble_model.predict(np.vstack(all_eval_features))
        accuracy = (predictions == np.hstack(all_eval_labels)).mean()
        logging.info(f"Ensemble Model Accuracy: {accuracy:.4f}")
    except Exception as e:
        logging.error(f"Error during ensemble model evaluation: {e}")
        raise

    return predictions, np.hstack(all_eval_labels), accuracy

In [14]:
# ==========================
# Training with GUI
# ==========================

def reshape_data_for_model(inputs, model):
    """Adjust the input shape to match the model's expected input shape."""
    input_size = inputs.view(inputs.size(0), -1).size(1)  # Flatten inputs
    logging.info(f"Current input size: {input_size}")

    try:
        # Check if the model has a fully connected layer as the first layer
        if hasattr(model, 'fc1'):
            model_input_size = model.fc1.in_features
        else:
            # Handle other model types that may not have fc1
            if hasattr(model, 'conv1'):
                model_input_size = None  # For convolutional models, we'll handle differently
            else:
                logging.error("Model type is not recognized for reshaping.")
                raise ValueError("Model does not have identifiable input features.")

    except Exception as e:
        logging.error(f"Error accessing model input size: {e}")
        raise

    if model_input_size is not None:
        if input_size != model_input_size:
            logging.info(f"Reshaping input from {input_size} to {model_input_size}.")
            try:
                inputs = inputs.view(inputs.size(0), model_input_size)  # Reshape inputs for fully connected layers
            except RuntimeError as reshape_error:
                logging.error(f"Failed to reshape input: {reshape_error}")
                raise
    else:
        # Handle convolutional model inputs
        if len(inputs.shape) == 2:  # If inputs are 2D, reshape to 4D
            logging.info(f"Reshaping 2D input to 4D for convolutional model.")
            height = width = int(input_size ** 0.5)  # Assuming square input for convolutional models
            inputs = inputs.view(inputs.size(0), 1, height, width)  # Reshape to (batch_size, channels, height, width)

    return inputs

def run_training_with_gui(train_loader, eval_loader, num_models, num_epochs, initial_learning_rate):
    """
    Initializes the GUI and orchestrates the training of multiple models in a separate thread to keep the GUI responsive.
    """
    root = tk.Tk()
    gui = TrainingGUI(root, num_models, num_epochs)

    regular_models = []
    mini_epoch_models = []

    # Initialize Helper Reinforcement Model
    helper = HyperparameterHelper(initial_lr=initial_learning_rate)

    def train_models():
        """
        Orchestrates the training of multiple models, handling errors with dynamic reshaping, preprocessing, and post-processing.
        Ensures graceful recovery from errors without breaking the process.
        """
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
        # Debug statement to log the data type of num_models
        logging.debug(f"Data type of num_models: {type(num_models)}")
    
        for model_num in range(1, num_models + 1):
            model = None
            model_type = random.choice(['CNN', 'DNN', 'ResNet', 'ViT'])
    
            try:
                model = get_random_model(model_type=model_type)
                model.to(device)
                logging.info(f"Training Model {model_num}/{num_models} with architecture: {model.__class__.__name__}")
    
                # Train the model
                train_regular_model(
                    model, train_loader, eval_loader, num_epochs, initial_learning_rate,
                    gui, model_num, num_models, helper, device
                )
    
                regular_models.append(model)
    
                # Post-process the model after training
                postprocessor = postprocessor_for_model(model)
                postprocessor.apply()
    
            except TypeError as type_error:
                logging.error(f"TypeError encountered with Model {model_num} ({model_type}): {type_error}")
                logging.error(f"Check types for model_num: {model_num} and num_models: {num_models}.")
    
            except Exception as model_error:
                logging.error(f"Error with Model {model_num} ({model_type}): {model_error}")

    logging.info("All regular models have been trained.")

    # Combine models using Random Forest
    ensemble_model = combine_models_with_error_handling(regular_models, train_loader)
    
    # Evaluate the ensemble model
    evaluate_ensemble_with_error_handling(ensemble_model, eval_loader, gui)

    # Notify GUI of completion
    gui.queue.put({'type': 'training_completed'})

        
    # Start training in a separate thread
    threading.Thread(target=train_models).start()
    root.mainloop()


def combine_models_with_error_handling(regular_models, train_loader):
    """Combine models into a Random Forest classifier with error handling."""
    try:
        logging.info("Combining models using Random Forest.")
        ensemble_model = combine_models_random_forest(train_loader, regular_models)
        return ensemble_model
    except Exception as e:
        logging.error(f"Error combining models: {e}")
        return None

def evaluate_ensemble_with_error_handling(ensemble_model, eval_loader, gui):
    """Evaluate the ensemble model with error handling."""
    try:
        _, _, ensemble_accuracy = evaluate_ensemble_model(ensemble_model, eval_loader)
        logging.info(f"Ensemble Model Accuracy: {ensemble_accuracy:.4f}")
        gui.queue.put({'type': 'ensemble_accuracy', 'accuracy': ensemble_accuracy})
    except Exception as e:
        logging.error(f"Error evaluating ensemble model: {e}")

In [ ]:
# ==========================
# Main Function
# ==========================

def main():
    """
    Main function to load data, prepare data loaders, set hyperparameters, initiate training, and evaluate the ensemble model.
    """
    # 1. Load ARC Data
    arc_data = load_arc_data()

    # 2. Prepare Training and Evaluation Data
    train_loader = prepare_training_data(arc_data)
    if train_loader is None:
        logging.error("Failed to prepare training data. Exiting.")
        return

    eval_loader = prepare_evaluation_data(arc_data)
    if eval_loader is None:
        logging.error("Failed to prepare evaluation data. Exiting.")
        return

    # 3. Set Hyperparameters (randomized)
    num_models, num_epochs, initial_learning_rate, accuracy_threshold = randomize_params()

    # 4. Run Training with GUI
    run_training_with_gui(train_loader, eval_loader, num_models, num_epochs, initial_learning_rate)

    # 5. After training, combine models using Random Forest
    # Note: This is now handled automatically within the training thread.

    # Placeholder for additional steps if needed
    # For example, further evaluation, saving ensemble model, etc.

if __name__ == '__main__':
    main()


[2024-10-15 18:01:51,472] INFO - Starting to load ARC dataset files...
[2024-10-15 18:01:51,495] INFO - Successfully loaded arc-agi_training-solutions from arc-agi_training_solutions.json on attempt 1.
[2024-10-15 18:01:51,528] INFO - Successfully loaded arc-agi_training-challenges from arc-agi_training_challenges.json on attempt 1.
[2024-10-15 18:01:51,535] INFO - Successfully loaded arc-agi_evaluation-solutions from arc-agi_evaluation_solutions.json on attempt 1.
[2024-10-15 18:01:51,574] INFO - Successfully loaded arc-agi_evaluation-challenges from arc-agi_evaluation_challenges.json on attempt 1.
[2024-10-15 18:01:51,582] INFO - Finished loading ARC dataset files. Total time: 0.11 seconds.
[2024-10-15 18:01:51,854] INFO - Preprocessed data: 1302 samples.
[2024-10-15 18:01:51,863] INFO - Created TensorDataset for training.
[2024-10-15 18:01:52,116] INFO - Preprocessed data: 1363 samples.
[2024-10-15 18:01:52,116] INFO - Created TensorDataset for evaluation.
[2024-10-15 18:01:52,121] 

Exception in thread Thread-4 (train_models):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2032.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\Users\Owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2032.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Owner\AppData\Local\Temp\ipykernel_17064\53583223.py", line 66, in train_models
TypeError: can only concatenate str (not "int") to str
